In [ ]:
import random
from collections import Counter
import pandas as pd
import ipywidgets as widgets 
import xlsxwriter
from datetime import date
import datetime
from collections import deque

In [2]:
def next_weekday(d, weekday):
    days_ahead = weekday - d.weekday()
    if days_ahead <= 0: # Target day already happened this week
        days_ahead += 7
    return d + datetime.timedelta(days_ahead)

In [3]:
def  reset_list():
    Employee_List = ["AB", "BC", "CD", "DE", "EF", "FG", "GH", "HI"]
    return Employee_List

In [4]:
def print_schedule(sched):
    for i in range(len(sched)):
        print(sched[i])

In [5]:
def time_off(form, day):
    if form.children[day].value != None:
        return list(form.children[day].value)
    return "None"

In [6]:
def assign(sched):
    
    
    find = find_empty(sched)
    if not find:
        return True
    else:
        row, col = find
        
    Employee_List = reset_list()
    random.shuffle(Employee_List)
    
    # removing analyst on tdays from list
    t_days = time_off(form, row)
    for t_day in t_days:
        if t_day != "None":
            Employee_List.remove(t_day)
    
    # Chooses two items at a time through the employee list. This is guess assignments for both products at once
    for choice1, choice2 in zip(Employee_List[0::2]*5, Employee_List[1::2]*5):
        if condition(sched, [choice1, choice2], row, col):
            if col == 4:
                sched[row][col][0] = choice1
                sched[row][col][1] = choice1
            else:
                sched[row][col][0] = choice1
                sched[row][col][1] = choice2
            
            if assign(sched):
                return True
            
            sched[row][col][0] = 0
            #assign(sched)
            
    return False

In [7]:
# List of all conditions when assigning to our schedule
def condition(sched, choice, day, shift):
    
    prdct1_choice = choice[0] 
    prdct2_choice = choice[1]
    
    
    # condition 1: employee cannot work both early and late shift
    if (shift == 5 and prdct1_choice in sched[day][0]) or (shift == 5 and prdct2_choice in sched[day][0]):
        return False
    # condition: employee cannot work early/late back to back days
    if (day > 0 and shift == 0 and prdct1_choice in sched[day-1][0]) or (day > 0 and shift == 0 and prdct2_choice in sched[day-1][0]):
        return False
    if (day > 0 and shift == 5 and prdct1_choice in sched[day-1][5]) or (day > 0 and shift == 5 and prdct2_choice in sched[day-1][5]):
        return False
    # cannot work two shifts back to back
    if (shift > 0 and prdct1_choice in sched[day][shift-1]) or (shift > 0 and prdct2_choice in sched[day][shift-1]):
        return False
    
    
    # Makes a list of all assignments
    new_list = []
    for i in range(len(sched[day])):
        for j in sched[day][i]:
            new_list.append(j)
        
    
    if (Counter(new_list).get(prdct1_choice) == None) or (Counter(new_list).get(prdct2_choice) == None):
        return True

    # Can only work 2 shifts a day
    # Check to see if guess has already appeared twice that day
    elif (Counter(new_list).get(prdct1_choice) > 2) or (Counter(new_list).get(prdct2_choice) > 2):
        return False
    
    
    return True

In [8]:
# find an unassigned shift spot
def find_empty(sched):
    for i in range(len(sched)):
        for j in range(len(sched[0])):
            if sched[i][j][0] == 0:
                return (i,j)    
    return None

In [9]:
# Dummy Schedule
sched = [[[0,0],[0,0],[0,0],[0,0],[0,0],[0,0]],
         [[0,0],[0,0],[0,0],[0,0],[0,0],[0,0]],
         [[0,0],[0,0],[0,0],[0,0],[0,0],[0,0]],
         [[0,0],[0,0],[0,0],[0,0],[0,0],[0,0]],
         [[0,0],[0,0],[0,0],[0,0],[0,0],[0,0]]]

# Employee List
Employee_List = ["AB", "BC", "CD", "DE", "EF", "FG", "GH", "HI"]

In [10]:
Options = Employee_List + ["None"]

print("Please select employee time off")

form_item_layout = widgets.Layout(
    display='flex',
    flex_flow='flex-wrap',
    justify_content='space-between'
)

form_items = [widgets.SelectMultiple(
    options=Options,
    value=['None'],
    #rows=10,
    description='Monday',
    disabled=False
), 
widgets.SelectMultiple(
    options=Options,
    value=['None'],
    #rows=10,
    description='Tuesday',
    disabled=False
),
widgets.SelectMultiple(
    options=Options,
    value=['None'],
    #rows=10,
    description='Wenesday',
    disabled=False
),
widgets.SelectMultiple(
    options=Options,
    value=['None'],
    #rows=10,
    description='Thursday',
    disabled=False
),
widgets.SelectMultiple(
    options=Options,
    value=['None'],
    #rows=10,
    description='Friday',
    disabled=False
)]

form = widgets.Box(form_items, layout=widgets.Layout(
    display='flex',
    flex_flow='row',
    border='solid 2px',
    align_items='stretch',
    width='1000px',
    height='130 ')) 
form

Please select employee time off


Box(children=(SelectMultiple(description='Monday', index=(7,), options=('JJB', 'DD', 'AM', 'MA', 'MH', 'NA', '…

In [11]:
while_count = 0
final_dict = {}
while len(final_dict) is 0 or final_dict[50]["Product1_Assignment"] is 0:
    if while_count > 10000:
        print("Failed: Too many people out for this week. Please reduce the number and adjust manually")
        break
    else:
        assign(sched)
        len_loop = 1
        while_count+=1
    for i in range(len(sched)):
        for j in range(len(sched[0])):
            if j in [2,3]:
                final_dict.update({len_loop:{"Product1_Assignment": sched[i][j][0], "Product2_Assignment": sched[i][j][1]}})
                len_loop+=1
            else:
                final_dict.update({len_loop:{"Product1_Assignment": sched[i][j][0], "Product2_Assignment": sched[i][j][1]}})
                len_loop+=1
                final_dict.update({len_loop:{"Product1_Assignment": sched[i][j][0], "Product2_Assignment": sched[i][j][1]}})
                len_loop+=1

In [14]:
Time_dict = {"AB": 0,
"BC": 0,
"CD": 0,
"DE": 0,
"EF": 0,
"FG": 0,
"GH": 0,
"HI": 0}

for i in range(len(sched)):
    for j in range(len(sched[0])):
        if j == 2 or j == 3:
            Time_dict[sched[i][j][0]] +=1
        else:
            Time_dict[sched[i][j][0]] +=2
            
for i in range(len(sched)):
    for j in range(len(sched[0])):
        if j == 2 or j == 3:
            Time_dict[sched[i][j][1]] +=1
        else:
            Time_dict[sched[i][j][1]] +=2


Time_df = pd.DataFrame.from_dict(Time_dict, orient="index", columns=["Hours on Shift"])

In [16]:
def queue_maker():
    q = deque()
    for i in range(1,len(final_dict)+1):
        q.append(final_dict[i]["Product2_Assignment"])
    return q

In [17]:
#Compile a list of everyone that took time off
time_off_dict = {}
for i in range(5):
    time_off_dict.update({i:list(form.children[i].value)})
time_off_dict 

#Laod data
Time_df 
#Create a Excel sheet and add the timetable dataframe
writer = pd.ExcelWriter('Demo.xlsx', engine='xlsxwriter')
Time_df.to_excel(writer, sheet_name='Assignments', startcol=13, startrow=19)

workbook  = writer.book
wsAssignment = writer.sheets['Assignments']
merge_format1 = workbook.add_format({'align': 'center',
                                   'valign':'vcenter',
                                   'fg_color': '#66FFCC',
                                   'text_wrap': 'true',
                                    'font_name': 'Avenir Next M for BBG',
                                    'border':True})
merge_format2 = workbook.add_format({'align': 'center',
                                    'font_name': 'Avenir Next M for BBG',
                                    'border':True})
merge_format3 = workbook.add_format({'align': 'center',
                                    'font_name': 'Avenir Next M for BBG',
                                    'bold':True,
                                    'border':True})
merge_format4 = workbook.add_format({'align':'center',
                                    'font_name': 'Avenir Next M for BBG',
                                    'fg_color': '#CCFFCC',
                                    'border':True})
merge_format5 = workbook.add_format({'align':'center',
                                    'font_name': 'Avenir Next M for BBG',
                                    'fg_color': '#CCFFFF',
                                     'border':True})
merge_format6 = workbook.add_format({'align':'center',
                                    'font_name': 'Avenir Next M for BBG',
                                    'fg_color': '#FFFFFF',
                                     'border':True})
#add analyst name key

Updated_List = ["AB", "BC", "CD", "DE", "EF", "FG", "GH", "HI"]
Updated_List_Full = ["Employee1", "Employee2","Employee3", "Employee4","Employee5", "Employee6","Employee7"]


for index, name in enumerate(Updated_List):
        wsAssignment.write(index+18, 0, name, merge_format3)
        
for index, full_name in enumerate(Updated_List_Full):
    wsAssignment.write(index+18, 1, full_name)

wsAssignment.set_column(0, 0, 10)
wsAssignment.merge_range('A1:A2', 'CACW SCHEDULE', merge_format1)

day_count = 0
for i in range(1,11):
    if (i % 2) != 0:
        if i == 1:
            wsAssignment.merge_range(0, i, 0, i+1, str((next_weekday(date.today(), 0).strftime("%d-%b"))), merge_format2)
            wsAssignment.merge_range(2, i, 2, i+1, '', workbook.add_format({'fg_color':'#FFCCFF'}))
            wsAssignment.write(1, i, 'Product2', workbook.add_format({'fg_color':'#CCFFCC'}))
            wsAssignment.write(1, i+1, 'Product1', workbook.add_format({'fg_color':'#CCFFFF'}))
            day_count +=1
        else:
            wsAssignment.merge_range(0, i, 0, i+1, str((next_weekday(date.today(), 0) + datetime.timedelta(int(day_count))).strftime("%d-%b")), merge_format2)
            wsAssignment.merge_range(2, i, 2, i+1, '', workbook.add_format({'fg_color':'#FFCCFF'}))
            wsAssignment.write(1, i, 'Product2', workbook.add_format({'fg_color':'#CCFFCC'}))
            wsAssignment.write(1, i+1, 'Product1', workbook.add_format({'fg_color':'#CCFFFF'}))
            day_count +=1
wsAssignment.write('A3', 'Scrum Master', workbook.add_format({'fg_color':'#FFCCFF','bold':True}))
for i in range(10):
    if i < 4:
        wsAssignment.write(i+3, 0, f'{i+8} to {i+9}', merge_format3)
    elif i ==4:
        wsAssignment.write(i+3, 0, f'{i+8} to {i-3}', merge_format3)
    else:
        wsAssignment.write(i+3, 0, f'{i-4} to {i-3}', merge_format3)
        
# M&A 
b=1
for i in range(1,55):
    if i in [7,8]:
        wsAssignment.write(i+2, 2, final_dict[b]["Product1_Assignment"], merge_format6)
        b+=1
    elif i < 11:
        wsAssignment.write(i+2, 2, final_dict[b]["Product1_Assignment"], merge_format5)
        b+=1
    if i in [18,19]:
        wsAssignment.write(i-9, 4, final_dict[b]["Product1_Assignment"], merge_format6)
        b+=1
    elif i in range(12,22):
        wsAssignment.write(i-9, 4, final_dict[b]["Product1_Assignment"], merge_format5)
        b+=1
    if i in [29,30]:
        wsAssignment.write(i-20, 6, final_dict[b]["Product1_Assignment"], merge_format6)
        b+=1
    elif i in range(23,33):
        wsAssignment.write(i-20, 6, final_dict[b]["Product1_Assignment"], merge_format5)
        b+=1
    if i in [40,41]:
        wsAssignment.write(i-31, 8, final_dict[b]["Product1_Assignment"], merge_format6)
        b+=1
    elif i in range(34,44):
        wsAssignment.write(i-31, 8, final_dict[b]["Product1_Assignment"], merge_format5)
        b+=1
c=41
d = 0
for i in range(1,11):
    if i in [7,8]:
        wsAssignment.write(3+d, 10, final_dict[c]["Product1_Assignment"], merge_format6)
        c+=1
        d+=1
    else:
        wsAssignment.write(3+d, 10, final_dict[c]["Product1_Assignment"], merge_format5)
        c+=1
        d+=1


q = queue_maker()

a = 1
for i in range(5):
    if i == 4:
        end = 11
    end = 11
    for j in range(1,end):
        if j in [7,8]:
            wsAssignment.write(j+2, a, q.popleft(), merge_format6)
        else:
            wsAssignment.write(j+2, a, q.popleft(), merge_format4) 
    a+=2
    
#for i in range(len(sched)):
#    for j in range(len(sched[0])):
        
column = "A"

wsAssignment.write('A14', 'Time Off', merge_format1)

for i in range(0,5):
    Pos1 = str(chr(ord(column) + 1) + "14")
    Pos2 = str(chr(ord(column) + 2) + "14")
    column = Pos2[0]
    if time_off_dict[i] != ['None']:
        wsAssignment.merge_range(f'{Pos1}:{Pos2}', str(time_off_dict[i]).strip("[]").strip("''"))
        column = Pos2[0]
        print(time_off_dict[i])
writer.save()